![sql](img/sql-logo.jpg)

# A beginner's guide to databases, SQL, & using them with `pandas`

**Scenario:** You are a data analyst for the Homeland Security, trying to create reports on the active airports world wide. The data you need to access is in a SQL database. YOu need to be able to query for the data in a database!

## Learning goals:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use `JOIN` to merge tables along logical columns
    - Grouping Data with SQL, inlcuding `HAVING`
- Goal 5: Convert SQL to pandas

## Goal 1: Summarize

To date the data we've seen has looked like [this.](https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Intakes/wter-evkm)

That is how we need data to look to run analysis and build models.<br>
But it doesn't _live_ there in it's native state.

[Netflix has a great article](https://medium.com/netflix-techblog/notebook-innovation-591ee3221233) describing three different data roles at their company, their different needs, and their toolsets.

![netflix](img/netflix-data-roles.jpeg)

Examining that graphic, SQL shows up as one of the tools of the _Data Engineer_ 

Data Engineers provide the essential data architecture services that make data science possible.

![hierarchy](img/ai-hierachy.png)

[Source: Monica Rogati’s fantastic Medium post “The AI Hierarchy of Needs”
](https://hackernoon.com/the-ai-hierarchy-of-needs-18f111fcc007)

## What does it mean to **Engineer Data**?

Let's start with a basic scenario:<br>
You are HR. You have *no* current database. 
How would you set it up?

Data needs considerations:
- What you want to store
- What "views" you anticipate wanting in the future

Structure considerations:
- Speed of retrieval
- How much data you are accessing
- How much you are storing

![etl](img/etl.png)

[img source: Jeff Hammerbacher’s slide from UC Berkeley CS 194 course ](https://bcourses.berkeley.edu/courses/1377158/pages/cs-194-16-introduction-to-data-science-fall-2015)

### What is a Relational Database? 

![rdb](img/relational-dbms-model.png)
[reference for image ](https://www.studytonight.com/dbms/database-model.php)

### POPULAR RDBMS

- SQLite
- MySQL
- PostgreSql
- Oracle DB
- SQL Server

***
### SQL Readings

[History of SQL](https://www.businessnewsdaily.com/5804-what-is-sql.html)

[The original SQL paper from the 1970s.](https://www.seas.upenn.edu/~zives/03f/cis550/codd.pdf)

***
## Goal 2: Database terminology

### Relational Database Schema

![schema](img/MySQL_Schema_Music_Example.png)

[source of image](https://database.guide/what-is-a-database-schema/)

***
### SQLite

<img src="img/SQLite-Python.jpg" width=500>

- C library (fast)
- lightweight disk-based database
- that doesn’t require a separate server process
- nonstandard variant of the SQL query language
- applications can use SQLite for internal data storage
- use case: prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle

[sqlite documentation](https://docs.python.org/2/library/sqlite3.html)



## But what about connecting to database servers?

To connect to other database types there are many different libraries:
- Oracle: [cx_Oracle](https://oracle.github.io/python-cx_Oracle/)
- MySQL: [MySQL-python](https://github.com/farcepest/MySQLdb1)
- PostgreSQL: [Psycopg2](http://initd.org/psycopg/docs/)
- Microsoft SQL Server: [pymssql](http://www.pymssql.org/en/stable/)

These all implement [PEP 249: DB API v2](https://www.python.org/dev/peps/pep-0249/)

## Goal 3: Get going with sqlite!

In [1]:
import sqlite3

#### Load a DB object with `connect` and `cursor`

In [2]:
!ls data

flights.db


In [3]:
!head data/flights.db

SQLite format 3   @     	  �                                                            	 -�        s�o �                �f�%tableroutesroutesqCREATE TABLE routes (
[index] INTEGER,
  [airline] TEXT,
  [airline_id] TEXT,
  [source] TEXT,
  [source_id] TEXT,
  [dest] TEXT,
  [dest_id] TEXT,
  [codeshare] TEXT,
  [stops] TEXT,


In [4]:
con = sqlite3.connect('data/flights.db')
cursor = con.cursor()

#### Use sqlite_master to find all the tables in the schema
Get the schema of a database from a db in sqlite

In [5]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('airports',), ('airlines',), ('routes',)]


In [6]:
cursor.description

(('name', None, None, None, None, None, None),)

In [7]:
cursor.execute("SELECT name, type FROM sqlite_master;")
print(cursor.fetchall())

[('airports', 'table'), ('ix_airports_index', 'index'), ('airlines', 'table'), ('ix_airlines_index', 'index'), ('routes', 'table'), ('ix_routes_index', 'index')]


In [8]:
cursor.execute("SELECT type, tbl_name, rootpage FROM sqlite_master;")
results = cursor.fetchall()
for res in results:
    print(res)

('table', 'airports', 2)
('index', 'airports', 3)
('table', 'airlines', 945)
('index', 'airlines', 946)
('table', 'routes', 1393)
('index', 'routes', 1394)


In [9]:
cursor.description

(('type', None, None, None, None, None, None),
 ('tbl_name', None, None, None, None, None, None),
 ('rootpage', None, None, None, None, None, None))

#### Get information about one table

**A note about** `execute`<br>
Each time you use it, you reset the value of cursor

In [10]:
cursor.execute("SELECT * FROM airports as a").fetchone()

(0,
 '1',
 'Goroka',
 'Goroka',
 'Papua New Guinea',
 'GKA',
 'AYGA',
 '-6.081689',
 '145.391881',
 '5282',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [11]:
cursor.fetchone()

(1,
 '2',
 'Madang',
 'Madang',
 'Papua New Guinea',
 'MAG',
 'AYMD',
 '-5.207083',
 '145.7887',
 '20',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [12]:
cursor.fetchmany(4)

[(2,
  '3',
  'Mount Hagen',
  'Mount Hagen',
  'Papua New Guinea',
  'HGU',
  'AYMH',
  '-5.826789',
  '144.295861',
  '5388',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (3,
  '4',
  'Nadzab',
  'Nadzab',
  'Papua New Guinea',
  'LAE',
  'AYNZ',
  '-6.569828',
  '146.726242',
  '239',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (4,
  '5',
  'Port Moresby Jacksons Intl',
  'Port Moresby',
  'Papua New Guinea',
  'POM',
  'AYPY',
  '-9.443383',
  '147.22005',
  '146',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (5,
  '6',
  'Wewak Intl',
  'Wewak',
  'Papua New Guinea',
  'WWK',
  'AYWK',
  '-3.583828',
  '143.669186',
  '19',
  '10',
  'U',
  'Pacific/Port_Moresby')]

#### Use description

In [13]:
cursor.description

(('index', None, None, None, None, None, None),
 ('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('code', None, None, None, None, None, None),
 ('icao', None, None, None, None, None, None),
 ('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None),
 ('altitude', None, None, None, None, None, None),
 ('offset', None, None, None, None, None, None),
 ('dst', None, None, None, None, None, None),
 ('timezone', None, None, None, None, None, None))

#### Or use `Pragma`
`Pragma` tool [link here](https://www.sqlite.org/pragma.html#pragma_table_info)

**output**<br>
`(column id, column name, data type, whether or not the column can be NULL, and the default value for the column)`

In [14]:
z = (1,2)
print(*z, sep ='\n')

1
2


In [15]:
cursor.execute("PRAGMA table_info(airports)")
info = cursor.fetchall()
print(*info, sep = "\n") 

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'city', 'TEXT', 0, None, 0)
(4, 'country', 'TEXT', 0, None, 0)
(5, 'code', 'TEXT', 0, None, 0)
(6, 'icao', 'TEXT', 0, None, 0)
(7, 'latitude', 'TEXT', 0, None, 0)
(8, 'longitude', 'TEXT', 0, None, 0)
(9, 'altitude', 'TEXT', 0, None, 0)
(10, 'offset', 'TEXT', 0, None, 0)
(11, 'dst', 'TEXT', 0, None, 0)
(12, 'timezone', 'TEXT', 0, None, 0)


#### Making fetch happen

`.fetchall()` is how you get the query results out of the object.

You can also `.fetchone()` or `.fetchmany()`

**Task:** Get the descriptive data for airlines and routes tables

In [16]:
cursor.execute("PRAGMA table_info(airlines)")
info = cursor.fetchall()
print(*info, sep = "\n")

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'alias', 'TEXT', 0, None, 0)
(4, 'iata', 'TEXT', 0, None, 0)
(5, 'icao', 'TEXT', 0, None, 0)
(6, 'callsign', 'TEXT', 0, None, 0)
(7, 'country', 'TEXT', 0, None, 0)
(8, 'active', 'TEXT', 0, None, 0)


In [17]:
cursor.execute("PRAGMA table_info(routes)")
info = cursor.fetchall()
print(*info, sep = "\n")

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'airline', 'TEXT', 0, None, 0)
(2, 'airline_id', 'TEXT', 0, None, 0)
(3, 'source', 'TEXT', 0, None, 0)
(4, 'source_id', 'TEXT', 0, None, 0)
(5, 'dest', 'TEXT', 0, None, 0)
(6, 'dest_id', 'TEXT', 0, None, 0)
(7, 'codeshare', 'TEXT', 0, None, 0)
(8, 'stops', 'TEXT', 0, None, 0)
(9, 'equipment', 'TEXT', 0, None, 0)


***
## Goal 4: Use basic SQL commands 

### The Structure of a SQL Query

<img src='img/sql_statement.jpg'/>

### Constructing SQL queries

**`SELECT`**:  The columns you want
  - options: `DISTINCT`, using `AS` to rename columns, single number aggregates

**`FROM`:** the source tables
  - options: also uses aliasing with `AS`; joining with `[LEFT|INNER|RIGHT|FULL] JOIN ___ [ON|USING]`

**`WHERE`**: your filters
  - options: comparators like `=` & `>=`; `BETWEEN`, `IN`, `LIKE` (with wildcards `%`); booleans like `AND`, `OR`, `NOT`

**`ORDER BY`**: sorting
  - options: `ASC` (default) and `DESC`

**`LIMIT`**:  # of rows to return (pair with `OFFSET`)

**Task**: 
- Select only active airlines in the UK from the airlines table
- Select the unique list of countries with airports

In [19]:
cursor.execute("""SELECT name FROM airlines
WHERE active='Y' AND country = 'United Kingdom'
ORDER BY lower(name)
""")
cursor.fetchall()

[('AD Aviation',),
 ('Air Cudlua',),
 ('Air Foyle',),
 ('Air Southwest',),
 ('Air Wales',),
 ('All Europe',),
 ('Astraeus',),
 ('Aurigny Air Services',),
 ('BA CityFlyer',),
 ('BBN-Airways',),
 ('bmi',),
 ('bmibaby',),
 ('British Airways',),
 ('British International Helicopters',),
 ('British Mediterranean Airways',),
 ('British Midland Regional',),
 ('CB Airways UK ( Interliging Flights )',),
 ('Crest Aviation',),
 ('Eastern Airways',),
 ('easyJet',),
 ('Excel Airways',),
 ('Excel Charter',),
 ('First Choice Airways',),
 ('Flightline',),
 ('Flybe',),
 ('Flyglobespan',),
 ('GB Airways',),
 ('Highland Airways',),
 ('Jc royal.britannica',),
 ('Jet2.com',),
 ('Kinloss Flying Training Unit',),
 ('Monarch Airlines',),
 ('MyTravel Airways',),
 ('Norfolk County Flight College',),
 ('Rainbow Air Euro',),
 ('Royal European Airlines',),
 ('ScotAirways',),
 ('Thomas Cook Airlines',),
 ('Thomsonfly',),
 ('Virgin Atlantic Airways',)]

In [20]:
cursor.execute('''SELECT country FROM airports
GROUP BY country
''')
cursor.fetchall()

[('Afghanistan',),
 ('Albania',),
 ('Algeria',),
 ('American Samoa',),
 ('Angola',),
 ('Anguilla',),
 ('Antarctica',),
 ('Antigua and Barbuda',),
 ('Argentina',),
 ('Armenia',),
 ('Aruba',),
 ('Australia',),
 ('Austria',),
 ('Azerbaijan',),
 ('Bahamas',),
 ('Bahrain',),
 ('Bangladesh',),
 ('Barbados',),
 ('Belarus',),
 ('Belgium',),
 ('Belize',),
 ('Benin',),
 ('Bermuda',),
 ('Bhutan',),
 ('Bolivia',),
 ('Bosnia and Herzegovina',),
 ('Botswana',),
 ('Brazil',),
 ('British Indian Ocean Territory',),
 ('British Virgin Islands',),
 ('Brunei',),
 ('Bulgaria',),
 ('Burkina Faso',),
 ('Burma',),
 ('Burundi',),
 ('Cambodia',),
 ('Cameroon',),
 ('Canada',),
 ('Cape Verde',),
 ('Cayman Islands',),
 ('Central African Republic',),
 ('Chad',),
 ('Chile',),
 ('China',),
 ('Christmas Island',),
 ('Cocos (Keeling) Islands',),
 ('Colombia',),
 ('Comoros',),
 ('Congo (Brazzaville)',),
 ('Congo (Kinshasa)',),
 ('Cook Islands',),
 ('Costa Rica',),
 ("Cote d'Ivoire",),
 ('Croatia',),
 ('Cuba',),
 ('Cyprus

In [34]:
cursor.execute('''
SELECT DISTINCT country
FROM airports
ORDER BY 1
''')
print(*cursor.fetchall(), sep = '\n')

('Afghanistan',)
('Albania',)
('Algeria',)
('American Samoa',)
('Angola',)
('Anguilla',)
('Antarctica',)
('Antigua and Barbuda',)
('Argentina',)
('Armenia',)
('Aruba',)
('Australia',)
('Austria',)
('Azerbaijan',)
('Bahamas',)
('Bahrain',)
('Bangladesh',)
('Barbados',)
('Belarus',)
('Belgium',)
('Belize',)
('Benin',)
('Bermuda',)
('Bhutan',)
('Bolivia',)
('Bosnia and Herzegovina',)
('Botswana',)
('Brazil',)
('British Indian Ocean Territory',)
('British Virgin Islands',)
('Brunei',)
('Bulgaria',)
('Burkina Faso',)
('Burma',)
('Burundi',)
('Cambodia',)
('Cameroon',)
('Canada',)
('Cape Verde',)
('Cayman Islands',)
('Central African Republic',)
('Chad',)
('Chile',)
('China',)
('Christmas Island',)
('Cocos (Keeling) Islands',)
('Colombia',)
('Comoros',)
('Congo (Brazzaville)',)
('Congo (Kinshasa)',)
('Cook Islands',)
('Costa Rica',)
("Cote d'Ivoire",)
('Croatia',)
('Cuba',)
('Cyprus',)
('Czech Republic',)
('Denmark',)
('Djibouti',)
('Dominica',)
('Dominican Republic',)
('East Timor',)
('Ecua

### SQL Joins

SQL joins can be used to both **add** data to a table and **remove** data from a table. 

<img src="img/venn.png" width=550>

**Task** Write a query that will join the latitude and longitude data from the airports table to the information on the routes table

In [24]:
import pandas as pd

In [27]:
cursor.execute('''SELECT 
                r.*,
                latitude AS source_lat,
                longitude AS source_lon
        FROM routes AS r
        JOIN airports AS a
        ON r.source_id = a.id
''')
df = pd.DataFrame(cursor.fetchall())
df.columns = [x[0] for x in cursor.description]
df.head()

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment,source_lat,source_lon
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2,43.449928,39.956589
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2,46.283333,48.006278
2,2,2B,410,ASF,2966,MRV,2962,None,0,CR2,46.283333,48.006278
3,3,2B,410,CEK,2968,KZN,2990,None,0,CR2,55.305836,61.503333
4,4,2B,410,CEK,2968,OVB,4078,None,0,CR2,55.305836,61.503333


In [ ]:
cursor.execute("""
SELECT r.*, 
		latitude AS source_lat, 
		longitude AS source_lon
FROM routes AS r
JOIN airports AS a on source_id = a.id
""")
print(*cursor.description, sep='\n')

### Grouping statements

Combines `select` and `group by` when you want aggregates by values

`select` `min(x)` ... `max()`, `sum()`, etc

`group by x`

**Task**<br>
- Which countries have the highest amount of active airlines?
- Which countries have the highest amount of inactive airlines?
- What about airports by timezones?

In [34]:
# Which countries have the highest amount of active airports?
cursor.execute("""SELECT country, COUNT(*)
FROM airports
GROUP BY country
ORDER BY 2 DESC
""")
print(*cursor.fetchall(), sep='\n')

('United States', 1697)
('Canada', 435)
('Germany', 321)
('Australia', 263)
('Russia', 249)
('France', 233)
('China', 219)
('Brazil', 213)
('United Kingdom', 210)
('India', 140)
('Japan', 131)
('Indonesia', 114)
('Argentina', 103)
('South Africa', 103)
('Mexico', 94)
('Italy', 92)
('Sweden', 86)
('Spain', 84)
('Iran', 81)
('Turkey', 77)
('Colombia', 72)
('Norway', 69)
('Philippines', 64)
('Greece', 60)
('Thailand', 58)
('Venezuela', 58)
('Papua New Guinea', 57)
('New Zealand', 56)
('Kenya', 55)
('Switzerland', 53)
('Finland', 51)
('Malaysia', 47)
('Pakistan', 46)
('Madagascar', 45)
('Algeria', 44)
('Burma', 43)
('Peru', 43)
('Maldives', 42)
('Netherlands', 42)
('Saudi Arabia', 42)
('Portugal', 39)
('Chile', 38)
('Ecuador', 38)
('Bahamas', 37)
('Congo (Kinshasa)', 36)
('French Polynesia', 36)
('Nepal', 36)
('Belgium', 34)
('Poland', 34)
('Ukraine', 33)
('Ethiopia', 32)
('Marshall Islands', 32)
('Taiwan', 32)
('South Korea', 31)
('Cuba', 30)
('Vietnam', 30)
('Austria', 29)
('Botswana', 2

In [38]:
# Which countries have the highest amount of inactive airports?
cursor.execute("""SELECT country, COUNT(*), active
FROM airlines
WHERE active = 'N'
GROUP BY country
ORDER BY 2 DESC
""")
print(*cursor.fetchall(), sep='\n')

('United States', 939, 'N')
('Mexico', 427, 'N')
('United Kingdom', 367, 'N')
('Canada', 284, 'N')
('Russia', 158, 'N')
('Spain', 142, 'N')
('France', 97, 'N')
('Germany', 94, 'N')
('South Africa', 81, 'N')
('Nigeria', 80, 'N')
('Ukraine', 76, 'N')
('Kazakhstan', 73, 'N')
('Italy', 72, 'N')
('Australia', 67, 'N')
('Sweden', 55, 'N')
('Sudan', 47, 'N')
('Switzerland', 46, 'N')
('China', 45, 'N')
('Netherlands', 43, 'N')
('Austria', 38, 'N')
('Egypt', 36, 'N')
('Angola', 35, 'N')
('Brazil', 35, 'N')
('Chile', 35, 'N')
('Colombia', 34, 'N')
('Mauritania', 32, 'N')
('Portugal', 32, 'N')
('Thailand', 32, 'N')
('Indonesia', 31, 'N')
('Kyrgyzstan', 29, 'N')
('Czech Republic', 27, 'N')
('Bulgaria', 25, 'N')
('Denmark', 25, 'N')
('Dominican Republic', 25, 'N')
('Japan', 25, 'N')
('Iran', 24, 'N')
('New Zealand', 24, 'N')
('Libya', 23, 'N')
('Norway', 23, 'N')
('Sierra Leone', 23, 'N')
('Turkey', 23, 'N')
('United Arab Emirates', 23, 'N')
('Pakistan', 22, 'N')
('Venezuela', 22, 'N')
('Belgium', 

In [39]:
# What about airports by timezones?
cursor.execute("""SELECT id, name, timezone
FROM airports
GROUP BY timezone
""")
print(*cursor.fetchall(), sep='\n')

('253', 'Abidjan Felix Houphouet Boigny Intl', 'Africa/Abidjan')
('248', 'Kotoka Intl', 'Africa/Accra')
('1107', 'Bole Intl', 'Africa/Addis_Ababa')
('207', 'Blida', 'Africa/Algiers')
('3966', 'Assab Intl', 'Africa/Asmera')
('1044', 'Senou', 'Africa/Bamako')
('888', 'Bangui M Poko', 'Africa/Bangui')
('1050', 'Banjul Intl', 'Africa/Banjul')
('1060', 'Cufar', 'Africa/Bissau')
('1013', 'Chileka Intl', 'Africa/Blantyre')
('883', 'Maya Maya', 'Africa/Brazzaville')
('1120', 'Bujumbura Intl', 'Africa/Bujumbura')
('1126', 'Alexandria Intl', 'Africa/Cairo')
('1064', 'Inezgane', 'Africa/Casablanca')
('1099', 'Fria', 'Africa/Conakry')
('1081', 'Ziguinchor', 'Africa/Dakar')
('1176', 'Arusha', 'Africa/Dar_es_Salaam')
('4119', 'Ambouli International Airport', 'Africa/Djibouti')
('896', 'Tiko', 'Africa/Douala')
('5672', 'Hassan I Airport', 'Africa/El_Aaiun')
('1059', 'Freetown Lungi', 'Africa/Freetown')
('877', 'Francistown', 'Africa/Gaborone')
('1001', 'J M Nkomo Intl', 'Africa/Harare')
('790', 'Alex

In [40]:
# It's always a good idea to close our connections when we're done
cursor.close()
con.close()

## Goal 5: Transfering from sqlite to pandas

In [41]:
import pandas as pd

In [42]:
pd_con = sqlite3.connect("data/flights.db")
df = pd.read_sql_query("select * from airports limit 5;", pd_con)
df

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby


**Task**: 
Convert one of the earlier queries in the lesson to a pandas data frame

In [44]:
# Which countries have the highest amount of active airports?
conn = sqlite3.connect("data/flights.db")
sql = '''
SELECT id, name, country
FROM airlines
'''
pd.read_sql_query(sql, conn)

,id,name,country
0,1,Private flight,None
1,2,135 Airways,United States
2,3,1Time Airline,South Africa
3,4,2 Sqn No 1 Elementary Flying Training School,United Kingdom
4,5,213 Flight Unit,Russia
5,6,223 Flight Unit State Airline,Russia
6,7,224th Flight Unit,Russia
7,8,247 Jet Ltd,United Kingdom
8,9,3D Aviation,United States
9,10,40-Mile Air,United States


In [46]:
pd_con.close()
conn.close()

## Bonus: SQL-like querying in Pandas

`.query()`

[query documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html)

In [ ]:
# Get data for an example
import pandas as pd
shelter_data=pd.read_csv('https://data.austintexas.gov/api/views/9t4d-g238/rows.csv?accessType=DOWNLOAD')

In [ ]:
max_data = shelter_data.query('Name == "Max"')
max_data.head()


In [ ]:
#vs
shelter_data[shelter_data['Name']=="Max"].head()

In [ ]:
shelter_data.query('`Animal Type` == "Dog"').head()